In [165]:
%pip install  --quiet numpy matplotlib torch

Note: you may need to restart the kernel to use updated packages.


In [166]:
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
import torch.optim as optim

In [167]:
gamma = torch.tensor(2.0)   # Curvature of the Utility Function
rho = torch.tensor(0.04)    # Discount Rate
A = torch.tensor(0.5)       # TFP
alpha = torch.tensor(0.36)  # Returns to Scale
delta = torch.tensor(0.05)  # Depreciation Rate of Capital

n_hidden = 3
n_neurons = 8
batch_size = 1000
number_epochs = 200000
lowest_iteration = 0
min_loss = 100000
learning_rate = 0.001

k_min = 0.1 # Lower bound of sample interval
k_max = 10.0 # Upper bound of sample interval

grid_size = 10000 # Plotting grid

vf_init_guess = -60 # Value Function Initial Guess

n_burned_iter = 0 # Ergodic distribution estimation

np.random.seed(40)
torch.manual_seed(40)
torch.autograd.set_detect_anomaly(True)

In [168]:
class Feed_Forward_NN(nn.Module):
    def __init__(self, n_hidden, n_neurons, init_guess):
        super(Feed_Forward_NN, self).__init__()
        
        layers = []
        # input layer
        layers.extend([(nn.Linear(1, n_neurons)), nn.Tanh()])

        # hidden layers
        for _ in range(n_hidden - 1):
            layers.extend([(nn.Linear(n_neurons, n_neurons)), nn.Tanh()])

        # output layer
        layers.extend([(nn.Linear(n_neurons, 1))])
        layers[-1].bias.data.fill_(init_guess)
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [169]:

# Set up neural network
VF = Feed_Forward_NN(n_hidden=n_hidden, n_neurons=n_neurons, init_guess=vf_init_guess)
CF = Feed_Forward_NN(n_hidden=n_hidden, n_neurons=n_neurons, init_guess=0)

# Define trainable network parameters
theta_vf = VF.parameters()
theta_cf = CF.parameters()

# Define optimizer
optimizer = optim.Adam(list(theta_vf) + list(theta_cf), lr=learning_rate)

In [170]:
def HJB(k_capital, V, C):
    output = V(k_capital)
    gradients = torch.ones_like(output)
    torch.autograd.backward(output, gradients)
    v_prime = k_capital.grad

    Y = A * torch.pow(k_capital, alpha)                             # Output
    I = Y - torch.exp(C(k_capital))                                 # Investment
    mu_k = I - delta * k_capital                                    # Capital drift
    U = torch.pow(torch.exp(C(k_capital)), 1-gamma) / (1-gamma)     # Utility

    HJB = U - rho * V(k_capital) + torch.multiply(torch.detach(v_prime), mu_k)
    return HJB

In [171]:
def ergodic_distribution(k_capital, V, C):
    for i in range(n_burned_iter):
        output = V(k_capital)
        gradients = torch.ones_like(output)
        torch.autograd.backward(output, gradients)
        v_prime = k_capital.grad

        v_prime_max = torch.max(v_prime, torch.tensor(1e-7))
        Y = A * torch.pow(k_capital, alpha)
        I = Y - torch.exp(C(k_capital))
        dk_dt = I - delta * k_capital
        dt = 0.1
        k_capital = k_capital + dt * dk_dt
    return k_capital 

# def boundary_condition_ergodic_1_step (k_capital, V, C):
#     v_prime = torch.autograd.grad(V(k_capital), k_capital)[0]
#     v_prime_max = torch.max(v_prime, torch.tensor(1e-7))		# dV/dk
#     Y = A * torch.pow(k_capital, alpha)			                # Output
#     C = torch.exp(C(k_capital))        			                # Consumption
#     I = Y - C 									                # Investment
#     dK_dt = I - delta * k_capital  				                # Capital drift
#     dt = 0.1
#     k_capital_t_plus_one = k_capital + dt * dK_dt

#     # Require k_min < k (t + 1) < kMax 
#     error_lower_bound =  torch.max(torch.tensor([k_min]) - k_capital_t_plus_one, 0)
#     error_upper_bound = torch.max(k_capital_t_plus_one - torch.tensor([k_max]), 0)
#     error = error_lower_bound + error_upper_bound
#     return error

def boundary_condition(k_capital, V, C):
    Y = A * torch.pow(k_capital, alpha)			# Output
    I = Y - torch.exp(C(k_capital)) 			# Investment
    dK_dt = I - delta * k_capital  				# Capital drift

    epsilon = 1                                 # Values close enough to 0 can't have decreasing capital

    error = torch.where((k_capital < epsilon) & (dK_dt < 0) , dK_dt, 0)
    return error

In [172]:
def C_error(k_capital, V, C):
	output = V(k_capital)
	gradients = torch.ones_like(output)
	torch.autograd.backward(output, gradients)
	v_prime = k_capital.grad
	v_prime_max = torch.max(v_prime, torch.tensor(1e-7))
	c_err = torch.pow(v_prime_max, (-1/gamma)) - torch.exp(C(k_capital))
	return c_err

In [173]:
def Loss(batchSize, k_min, k_max):
    k_capital = torch.rand(batchSize, 1) * (k_max - k_min) + k_min
    ergodic_k_capital = ergodic_distribution(k_capital, VF, CF)
    ergodic_k_capital.requires_grad = True
    errorV = HJB(ergodic_k_capital, VF, CF)
    errorC = C_error(ergodic_k_capital, VF, CF)
    errorB = boundary_condition(ergodic_k_capital, VF, CF)

    lossV = torch.mean(torch.square(errorV))
    lossC = torch.mean(torch.square(errorC))
    lossB = torch.mean(torch.square(errorB))
    total_loss = lossV + lossC + lossB
    return lossV, lossC, lossB, total_loss

In [174]:
def training_step():
    optimizer.zero_grad()

    lossV, lossC, lossB, total_loss = Loss(batch_size, k_min, k_max)
    total_loss.backward()

    optimizer.step()

    return lossV, lossC, lossB, total_loss

def train_model(epochs, min_loss):
    losses_v = []
    losses_c = []
    losses_b = []
    total_losses = []

    best_vf = deepcopy(VF)
    best_cf = deepcopy(CF)
    lowest_iteration = 0
    min_loss = float("inf")

    for epoch in range(epochs):
        lossV, lossC, lossB, total_loss = training_step()

        if total_loss < min_loss:
            lowest_iteration = epoch
            min_loss = total_loss
            best_vf.load_state_dict(VF.state_dict())
            best_vf.load_state_dict(CF.state_dict())
            print(f"\nbest loss: {min_loss:.4f}", end="\r")

        losses_v.append(lossV.item())
        losses_c.append(lossC.item())
        losses_b.append(lossB.item())
        total_losses.append(total_loss.item())

    return losses_v, losses_c, losses_b, total_losses, lowest_iteration, min_loss, best_vf, best_cf

In [175]:
# Run Model (and output loss evolution)
lossesV, lossesC, lossesB, total_losses, min_loss_iteration, minimum_loss, best_VF, best_CF = train_model(number_epochs, min_loss)

VF.load_state_dict(best_VF.state_dict())
CF.load_state_dict(best_CF.state_dict())

print('\n')
print("Value error: ", lossesV[-1])
print("Consumption error : ", lossesC[-1])
print("Boundary error : ", lossesB[-1])
print("Sum of errors: ", total_losses[-1])

/if/research-gms/William/jesus/.venv/lib/python3.9/site-packages/torch/autograd/__init__.py:200: UserWarning: Error detected in MulBackward0. Traceback of forward call that caused the error:
  File "/apps/Anaconda/2022.10/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/apps/Anaconda/2022.10/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/if/research-gms/William/jesus/.venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/if/research-gms/William/jesus/.venv/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/if/research-gms/William/jesus/.venv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 728, in start
    self.io_loop.start()
  File "/if/research-gms/William/jesus/.venv/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start
  

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [1000, 1]] is at version 1; expected version 0 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!